# Install

In [2]:
# ! pip install -U agno
# ! pip install -U  python-dotenv rich
# ! pip install -U openai groq
# ! pip install sqlalchemy
# !pip install agno pandas
# stop

# Load Secrets

In [4]:
# Import necessary libraries
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the API keys from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
gro_api_key = os.getenv('Groq_API_KEY')


os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["GROQ_API_KEY"] = gro_api_key

# Print the keys (optional, for testing purposes)
print(f"OpenAI API Key: {bool(openai_api_key)}")
print(f"Gro API Key: {bool(gro_api_key)}")


OpenAI API Key: True
Gro API Key: True


# Create Databases with SQLAlchemy

In [6]:
import os
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Text, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
import bcrypt
from datetime import datetime
import sqlite3

# Define the database path
DATABASE_PATH = "./database/personality_test.db"

# First, close any existing connections
def close_existing_connections():
    try:
        # Try to connect to check if the database is in use
        conn = sqlite3.connect(DATABASE_PATH)
        conn.close()
        return True
    except sqlite3.Error as e:
        print(f"Database is in use: {e}")
        return False

# Remove existing database if it exists and is not in use
if os.path.exists(DATABASE_PATH):
    if close_existing_connections():
        try:
            os.remove(DATABASE_PATH)
            print(f"Removed existing database: {DATABASE_PATH}")
        except PermissionError:
            print("Could not remove database file - it may be locked by another process")
            # Continue anyway - we'll try to overwrite it
    else:
        print("Database is in use by another process - will attempt to overwrite")

# Create database directory if it doesn't exist
os.makedirs("./database", exist_ok=True)

# Create SQLAlchemy engine
engine = create_engine(f"sqlite:///{DATABASE_PATH}", echo=True)
Base = declarative_base()

# Define User model
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    password_hash = Column(String(60), nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    sessions = relationship("Session", back_populates="user", cascade="all, delete-orphan")
    assessment_results = relationship("AssessmentResult", back_populates="user", cascade="all, delete-orphan")

# Define Session model
class Session(Base):
    __tablename__ = 'sessions'
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    session_id = Column(String(36), unique=True, nullable=False)
    started_at = Column(DateTime, default=datetime.utcnow)
    completed_at = Column(DateTime, nullable=True)
    status = Column(String(20), default='active')
    
    # Relationships
    user = relationship("User", back_populates="sessions")
    assessment_results = relationship("AssessmentResult", back_populates="session", cascade="all, delete-orphan")

# Define AssessmentResult model
class AssessmentResult(Base):
    __tablename__ = 'assessment_results'
    
    id = Column(Integer, primary_key=True)
    session_id = Column(Integer, ForeignKey('sessions.id'), nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    question_id = Column(String(50), nullable=False)
    user_answer = Column(Text, nullable=False)
    validation_score = Column(Integer, nullable=True)
    personality_traits = Column(Text, nullable=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    session = relationship("Session", back_populates="assessment_results")
    user = relationship("User", back_populates="assessment_results")

# Create all tables
Base.metadata.create_all(engine)

# Create a session to interact with the database
SessionLocal = sessionmaker(bind=engine)
db_session = SessionLocal()

# Function to hash passwords
def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()).decode('utf-8')

# Create users if they don't exist
def create_initial_users():
    users_data = [
        {"username": "Nayeem", "password": "password"},
        {"username": "NoMan", "password": "password"}
    ]
    
    for user_data in users_data:
        existing_user = db_session.query(User).filter_by(username=user_data["username"]).first()
        if not existing_user:
            hashed_password = hash_password(user_data["password"])
            new_user = User(username=user_data["username"], password_hash=hashed_password)
            db_session.add(new_user)
    
    db_session.commit()
    print("Database created and initial users added successfully!")

# Execute the user creation
try:
    create_initial_users()

    # Verify the users were created
    users = db_session.query(User).all()
    print("\nUsers in database:")
    for user in users:
        print(f"ID: {user.id}, Username: {user.username}, Created: {user.created_at}")

except Exception as e:
    print(f"An error occurred: {e}")
    db_session.rollback()
finally:
    # Close the session
    db_session.close()
    # Dispose the engine to release all connections
    engine.dispose()

Removed existing database: ./database/personality_test.db
2025-09-08 13:15:19,843 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-08 13:15:19,844 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-09-08 13:15:19,844 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:15:19,846 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-09-08 13:15:19,846 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:15:19,849 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sessions")
2025-09-08 13:15:19,849 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:15:19,851 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sessions")
2025-09-08 13:15:19,851 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:15:19,852 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("assessment_results")
2025-09-08 13:15:19,853 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-08 13:15:19,854 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("assessmen

# Using Session and Agents

In [19]:
# Import all necessary modules
import os
import uuid
import json
import bcrypt
from datetime import datetime
from typing import Dict
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.memory.v2 import Memory
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.storage.sqlite import SqliteStorage
from pydantic import BaseModel

# Database setup
DATABASE_PATH = "./database/personality_test.db"
engine = create_engine(f"sqlite:///{DATABASE_PATH}")
SessionLocal = sessionmaker(bind=engine)
Base = declarative_base()

# Define models
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    password_hash = Column(String(60), nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    sessions = relationship("Session", back_populates="user")
    assessment_results = relationship("AssessmentResult", back_populates="user")

class Session(Base):
    __tablename__ = 'sessions'
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    session_id = Column(String(36), unique=True, nullable=False)
    started_at = Column(DateTime, default=datetime.utcnow)
    completed_at = Column(DateTime, nullable=True)
    status = Column(String(20), default='active')
    
    # Relationships
    user = relationship("User", back_populates="sessions")
    assessment_results = relationship("AssessmentResult", back_populates="session")

class AssessmentResult(Base):
    __tablename__ = 'assessment_results'
    
    id = Column(Integer, primary_key=True)
    session_id = Column(Integer, ForeignKey('sessions.id'), nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    question_id = Column(String(50), nullable=False)
    question_text = Column(Text, nullable=False)
    user_answer = Column(Text, nullable=False)
    positive_traits = Column(Text, nullable=True)
    negative_traits = Column(Text, nullable=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    session = relationship("Session", back_populates="assessment_results")
    user = relationship("User", back_populates="assessment_results")

# Create tables if they don't exist
Base.metadata.create_all(engine)


# Personality questionnaire
personality_questionnaire = [
    {
        'id': 'openness_1',
        'question': 'How do you usually approach new or unfamiliar situations?',
        'follow_up_prompts': [
            'Could you tell me more about a specific time you faced something new?',
            'What emotions do you typically feel when encountering unfamiliar experiences?'
        ]
    },
    {
        'id': 'conscientiousness_1',
        'question': 'How do you organize your work or personal projects?',
        'follow_up_prompts': [
            'Can you describe your system for tracking tasks and deadlines?',
            'How do you prioritize when multiple projects need your attention?'
        ]
    },
    {
        'id': 'extraversion_1',
        'question': 'What role do social interactions play in your life?',
        'follow_up_prompts': [
            'How do you typically feel after attending large social gatherings?',
            'Can you describe your ideal balance between social time and alone time?'
        ]
    },
]

# Structured output model
class PersonalityAssessment(BaseModel):
    question: str
    answer: str
    positive_traits: Dict[str, int]
    negative_traits: Dict[str, int]

# Session management class
class SessionManager:
    def __init__(self):
        self.active_sessions = {}
        self.db_session = SessionLocal()
    
    def create_session(self, user_id):
        session_id = str(uuid.uuid4())
        
        # Store session in database
        new_session = Session(
            user_id=user_id,
            session_id=session_id,
            started_at=datetime.utcnow(),
            status='active'
        )
        self.db_session.add(new_session)
        self.db_session.commit()
        
        # Initialize session state
        self.active_sessions[session_id] = {
            'db_session_id': new_session.id,
            'user_id': user_id,
            'start_time': datetime.utcnow(),
            'current_question': 0,
            'answers': {},
            'status': 'active'
        }
        
        return session_id
    
    def get_session(self, session_id):
        return self.active_sessions.get(session_id)
    
    def update_session(self, session_id, updates):
        if session_id in self.active_sessions:
            self.active_sessions[session_id].update(updates)
            
            # Update database if session status changed
            if 'status' in updates:
                session_obj = self.db_session.query(Session).filter_by(session_id=session_id).first()
                if session_obj:
                    session_obj.status = updates['status']
                    session_obj.completed_at = datetime.utcnow() if updates['status'] == 'completed' else None
                    self.db_session.commit()
    
    def close(self):
        self.db_session.close()

# Authentication functions
def authenticate_user(username, password):
    db_session = SessionLocal()
    user = db_session.query(User).filter_by(username=username).first()
    db_session.close()
    
    if user and bcrypt.checkpw(password.encode(), user.password_hash.encode()):
        return user.id
    return None

def create_user(username, password):
    db_session = SessionLocal()
    
    # Check if user already exists
    existing_user = db_session.query(User).filter_by(username=username).first()
    if existing_user:
        db_session.close()
        return False, "User already exists"
    
    # Create new user
    hashed_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()
    new_user = User(username=username, password_hash=hashed_password)
    db_session.add(new_user)
    db_session.commit()
    
    user_id = new_user.id
    db_session.close()
    
    return True, user_id

# Authentication menu
def auth_menu():
    print("Personality Assessment System")
    print("=" * 30)
    print("1. Login")
    print("2. Register")
    print("3. Exit")
    
    choice = input("Please choose an option (1-3): ")
    
    if choice == "1":
        username = input("Username: ")
        password = input("Password: ")
        
        user_id = authenticate_user(username, password)
        if user_id:
            return user_id, username
        else:
            print("Authentication failed. Please check your credentials.")
            return None, None
            
    elif choice == "2":
        username = input("Choose a username: ")
        password = input("Choose a password: ")
        
        success, result = create_user(username, password)
        if success:
            print(f"Account created successfully! User ID: {result}")
            return result, username
        else:
            print(f"Account creation failed: {result}")
            return None, None
            
    elif choice == "3":
        print("Goodbye!")
        return "exit", None
        
    else:
        print("Invalid choice.")
        return None, None

# Agent creation functions
def create_conversational_agent(user_id, session_id):
    # Create SqliteStorage for agent
    storage = SqliteStorage(
        table_name="agent_conversations",
        db_url=f"sqlite:///{DATABASE_PATH}"
    )
    
    # Create memory for agent
    memory_db = SqliteMemoryDb(
        table_name="agent_memory",
        db_url=f"sqlite:///{DATABASE_PATH}"
    )
    
    memory = Memory(
        db=memory_db,
        # user_id=str(user_id),
        # session_id=session_id
    )
    
    # Create the agent
    agent = Agent(
        model=OpenAIChat(id="gpt-4o", api_key=os.getenv('OPENAI_API_KEY')),
        name="Personality Interviewer",
        role="Engage users in friendly conversation to conduct personality assessment through structured questions",
        storage=storage,
        memory=memory,
        instructions=[
            "Maintain a friendly, engaging conversation style",
            "Ask one question at a time from the personality assessment questionnaire",
            "If an answer lacks context, ask thoughtful follow-up questions",
            "Remember the user's previous answers throughout the conversation",
            "After receiving a satisfactory answer, acknowledge it and move to the next question",
            "After completing all questions, summarize key insights and thank the user"
        ],
        markdown=True,
    )
    
    return agent

def create_validation_agent():
    # Create SQLiteStorage for agent
    storage = SqliteStorage(
        table_name="validation_results",
        db_url=f"sqlite:///{DATABASE_PATH}"
    )
    
    # Create the agent with structured output
    agent = Agent(
        model=OpenAIChat(id="gpt-4o", api_key=os.getenv('OPENAI_API_KEY')),
        name="Personality Validator",
        role="Validate user responses and provide structured personality assessment results",
        storage=storage,
        instructions=[
            "Analyze user responses for personality traits based on psychological frameworks like Big Five",
            "Identify positive traits and score them from 0 to 100",
            "Identify negative traits and score them from 0 to 100",
            "Return structured output with question, answer, positive_traits, and negative_traits",
            "Example format for traits: {'openness': 85, 'conscientiousness': 70}",
            "Be objective and evidence-based in your assessment"
        ],
        structured_outputs=PersonalityAssessment,
        markdown=True
    )
    
    return agent

# Main application functions
def run_personality_test(user_id, username):
    # Initialize session manager
    session_manager = SessionManager()
    
    # Create a new session
    session_id = session_manager.create_session(user_id)
    print(f"Welcome {username}! Session started: {session_id[:8]}...")
    
    # Create agents
    conv_agent = create_conversational_agent(user_id, session_id)
    val_agent = create_validation_agent()
    
    # Get session state
    session_state = session_manager.get_session(session_id)
    
    print("\nLet's begin the personality assessment. Please answer the following questions honestly.")
    print("You can type 'quit' at any time to exit the assessment.\n")
    
    # Main question loop
    for question_index, question_data in enumerate(personality_questionnaire):
        print(f"Question {question_index + 1}/{len(personality_questionnaire)}:")
        print(f"{question_data['question']}")
        
        satisfactory_answer = False
        full_answer = ""
        
        while not satisfactory_answer:
            # Get user input
            user_input = input("\nYour answer: ")
            
            if user_input.lower() == 'quit':
                print("Assessment cancelled. Your progress has been saved.")
                session_manager.update_session(session_id, {'status': 'cancelled'})
                session_manager.close()
                return
            
            # Add to full answer
            if full_answer:
                full_answer += " " + user_input
            else:
                full_answer = user_input
            
            # Check answer completeness using the conversational agent
            completeness_check = conv_agent.run(
                f"Assess if this answer is complete for a personality assessment: {user_input}. " +
                f"Question: {question_data['question']}. " +
                "Respond only with 'COMPLETE' or 'INCOMPLETE'."
            )
            
            if "INCOMPLETE" in completeness_check.content:  # Add .content here
                # Ask follow-up question
                follow_up = question_data['follow_up_prompts'][0]
                print(f"\nFollow-up: {follow_up}")
            else:
                satisfactory_answer = True
                
                # Get structured assessment from validation agent
                assessment = val_agent.run(
                    f"Question: {question_data['question']}\n" +
                    f"Answer: {full_answer}\n" +
                    "Please provide a structured personality assessment with positive and negative traits."
                )
                
                # Store the structured result
                db_session = SessionLocal()
                result = AssessmentResult(
                    session_id=session_state['db_session_id'],
                    user_id=user_id,
                    question_id=question_data['id'],
                    question_text=question_data['question'],
                    user_answer=full_answer,
                    positive_traits=json.dumps(assessment.content.positive_traits),
                    negative_traits=json.dumps(assessment.content.negative_traits),
                    created_at=datetime.utcnow()
                )
                db_session.add(result)
                db_session.commit()
                db_session.close()
                
                print(f"\n✓ Answer recorded.")
                print("Positive traits identified:")
                for trait, score in assessment.positive_traits.items():
                    print(f"  - {trait}: {score}/100")
                
                print("Areas for development:")
                for trait, score in assessment.negative_traits.items():
                    print(f"  - {trait}: {score}/100")
                
                print("Moving to the next question...\n")
    
    # Complete the session
    session_manager.update_session(session_id, {'status': 'completed'})
    session_manager.close()
    
    print("Assessment complete! Thank you for your participation.")
    print("Your results have been saved and will be available for review.")

def view_assessment_results(user_id):
    db_session = SessionLocal()
    
    # Get all sessions for the user
    sessions = db_session.query(Session).filter_by(user_id=user_id).all()
    
    print(f"\nAssessment History for User ID: {user_id}")
    print("=" * 50)
    
    for session in sessions:
        print(f"\nSession: {session.session_id} ({session.started_at}) - Status: {session.status}")
        
        # Get all results for this session
        results = db_session.query(AssessmentResult).filter_by(session_id=session.id).all()
        
        for result in results:
            print(f"\nQuestion: {result.question_text}")
            print(f"Answer: {result.user_answer}")
            
            if result.positive_traits:
                positive_traits = json.loads(result.positive_traits)
                print("Positive traits:")
                for trait, score in positive_traits.items():
                    print(f"  - {trait}: {score}/100")
            
            if result.negative_traits:
                negative_traits = json.loads(result.negative_traits)
                print("Areas for development:")
                for trait, score in negative_traits.items():
                    print(f"  - {trait}: {score}/100")
            
            print("-" * 30)
    
    db_session.close()

# Main application loop
def main():
    while True:
        user_id, username = auth_menu()
        
        if user_id == "exit":
            break
        elif user_id:
            # Ask if user wants to take a new test or view previous results
            choice = input("Would you like to (1) take a new personality test or (2) view previous results? ")
            
            if choice == "1":
                run_personality_test(user_id, username)
            elif choice == "2":
                view_assessment_results(user_id)
            else:
                print("Invalid choice.")
        
        print()  # Add a blank line for readability

# Run the application
if __name__ == "__main__":
    main()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16156\4150864184.py:22: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Personality Assessment System
1. Login
2. Register
3. Exit


Please choose an option (1-3):  1
Username:  Nayeem
Password:  password
Would you like to (1) take a new personality test or (2) view previous results?  1


C:\Users\lenovo\AppData\Local\Temp\ipykernel_16156\4150864184.py:120: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  started_at=datetime.utcnow(),
C:\Users\lenovo\AppData\Local\Temp\ipykernel_16156\4150864184.py:130: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'start_time': datetime.utcnow(),


Welcome Nayeem! Session started: faa318dd...

Let's begin the personality assessment. Please answer the following questions honestly.
You can type 'quit' at any time to exit the assessment.

Question 1/3:
How do you usually approach new or unfamiliar situations?



Your answer:  I keep calm



Follow-up: Could you tell me more about a specific time you faced something new?



Your answer:  Sure! A specific time I faced something new was when I was first introduced to interacting with images. Before, I could only process and respond to text, but now I have the capability to analyze and interpret images as well. It was an exciting shift, and it allowed me to explore new ways of helping users by understanding visuals and offering insights beyond text-based responses. It’s like adding a new layer to my capabilities!


AttributeError: 'str' object has no attribute 'positive_traits'

In [ ]:
stop

# Agno Agents with Memory and Session Management

In [ ]:
import json
from typing import Dict
from pydantic import BaseModel
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# Define structured output model
class PersonalityAssessment(BaseModel):
    question: str
    answer: str
    positive_traits: Dict[str, int]  # trait_name: score (0-100)
    negative_traits: Dict[str, int]  # trait_name: score (0-100)

# Update the validation agent to use structured output
def create_validation_agent():
    # Create SQLiteStorage for agent
    storage = SQLiteStorage(
        table_name="validation_results",
        db_url=f"sqlite:///{DATABASE_PATH}"
    )
    
    # Create the agent with structured output
    agent = Agent(
        model=OpenAIChat(id="gpt-4o"),
        name="Personality Validator",
        role="Validate user responses and provide structured personality assessment results",
        storage=storage,
        instructions=[
            "Analyze user responses for personality traits based on psychological frameworks like Big Five",
            "Identify positive traits and score them from 0 to 100",
            "Identify negative traits and score them from 0 to 100",
            "Return structured output with question, answer, positive_traits, and negative_traits",
            "Example format for traits: {'openness': 85, 'conscientiousness': 70}",
            "Be objective and evidence-based in your assessment"
        ],
        structured_outputs=PersonalityAssessment,
        markdown=True
    )
    
    return agent

# Update the assessment result model to store structured data
class AssessmentResult(Base):
    __tablename__ = 'assessment_results'
    
    id = Column(Integer, primary_key=True)
    session_id = Column(Integer, ForeignKey('sessions.id'), nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    question_id = Column(String(50), nullable=False)
    question_text = Column(Text, nullable=False)  # Store the exact question
    user_answer = Column(Text, nullable=False)
    positive_traits = Column(Text, nullable=True)  # JSON string of positive traits
    negative_traits = Column(Text, nullable=True)  # JSON string of negative traits
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    session = relationship("Session", back_populates="assessment_results")
    user = relationship("User", back_populates="assessment_results")

# Update the personality test execution to use structured output
def run_personality_test():
    print("Welcome to the Personality Assessment System!")
    print("=" * 50)
    
    # Authentication
    username = input("Username: ")
    password = input("Password: ")
    
    user_id = authenticate_user(username, password)
    if not user_id:
        print("Authentication failed. Please check your credentials.")
        return
    
    # Initialize database session and session manager
    db_session = SessionLocal()
    session_manager = SessionManager(db_session)
    
    # Create a new session
    session_id = session_manager.create_session(user_id)
    print(f"Session started: {session_id}")
    
    # Create agents
    conv_agent = create_conversational_agent(user_id, session_id)
    val_agent = create_validation_agent()
    
    # Get session state
    session_state = session_manager.get_session(session_id)
    
    print("\nLet's begin the personality assessment. Please answer the following questions honestly.")
    print("You can type 'quit' at any time to exit the assessment.\n")
    
    # Main question loop
    for question_index, question_data in enumerate(personality_questionnaire):
        print(f"Question {question_index + 1}/{len(personality_questionnaire)}:")
        print(f"{question_data['question']}")
        
        satisfactory_answer = False
        conversation_history = []
        full_answer = ""
        
        while not satisfactory_answer:
            # Get user input
            user_input = input("\nYour answer: ")
            
            if user_input.lower() == 'quit':
                print("Assessment cancelled. Your progress has been saved.")
                session_manager.update_session(session_id, {'status': 'cancelled'})
                db_session.close()
                return
            
            # Add to conversation history and full answer
            conversation_history.append(f"User: {user_input}")
            if full_answer:
                full_answer += " " + user_input
            else:
                full_answer = user_input
            
            # Check answer completeness using the conversational agent
            completeness_check = conv_agent.run(
                f"Assess if this answer is complete for a personality assessment: {user_input}. " +
                f"Question: {question_data['question']}. " +
                "Respond only with 'COMPLETE' or 'INCOMPLETE'."
            )
            
            if "INCOMPLETE" in completeness_check:
                # Ask follow-up question
                follow_up = question_data['follow_up_prompts'][0]  # Use first follow-up for simplicity
                print(f"\nFollow-up: {follow_up}")
            else:
                satisfactory_answer = True
                
                # Get structured assessment from validation agent
                assessment: PersonalityAssessment = val_agent.run(
                    f"Question: {question_data['question']}\n" +
                    f"Answer: {full_answer}\n" +
                    "Please provide a structured personality assessment with positive and negative traits."
                )
                
                # Store the structured result
                from models import AssessmentResult
                result = AssessmentResult(
                    session_id=session_state['user_id'],  # This should be the session ID from the database
                    user_id=user_id,
                    question_id=question_data['id'],
                    question_text=question_data['question'],
                    user_answer=full_answer,
                    positive_traits=json.dumps(assessment.positive_traits),
                    negative_traits=json.dumps(assessment.negative_traits),
                    created_at=datetime.utcnow()
                )
                db_session.add(result)
                db_session.commit()
                
                print(f"\n✓ Answer recorded.")
                print("Positive traits identified:")
                for trait, score in assessment.positive_traits.items():
                    print(f"  - {trait}: {score}/100")
                
                print("Areas for development:")
                for trait, score in assessment.negative_traits.items():
                    print(f"  - {trait}: {score}/100")
                
                print("Moving to the next question...\n")
    
    # Complete the session
    session_manager.update_session(session_id, {'status': 'completed'})
    db_session.close()
    
    print("Assessment complete! Thank you for your participation.")
    print("Your results have been saved and will be available for review.")

# Function to view assessment results
def view_assessment_results(user_id):
    db_session = SessionLocal()
    
    # Get all sessions for the user
    sessions = db_session.query(Session).filter_by(user_id=user_id).all()
    
    print(f"\nAssessment History for User ID: {user_id}")
    print("=" * 50)
    
    for session in sessions:
        print(f"\nSession: {session.session_id} ({session.started_at}) - Status: {session.status}")
        
        # Get all results for this session
        results = db_session.query(AssessmentResult).filter_by(session_id=session.id).all()
        
        for result in results:
            print(f"\nQuestion: {result.question_text}")
            print(f"Answer: {result.user_answer}")
            
            if result.positive_traits:
                positive_traits = json.loads(result.positive_traits)
                print("Positive traits:")
                for trait, score in positive_traits.items():
                    print(f"  - {trait}: {score}/100")
            
            if result.negative_traits:
                negative_traits = json.loads(result.negative_traits)
                print("Areas for development:")
                for trait, score in negative_traits.items():
                    print(f"  - {trait}: {score}/100")
            
            print("-" * 30)
    
    db_session.close()

# Update the main execution block
if __name__ == "__main__":
    # First authenticate
    username = input("Username: ")
    password = input("Password: ")
    
    user_id = authenticate_user(username, password)
    if not user_id:
        print("Authentication failed.")
    else:
        # Ask if user wants to take a new test or view previous results
        choice = input("Would you like to (1) take a new personality test or (2) view previous results? ")
        
        if choice == "1":
            run_personality_test()
        elif choice == "2":
            view_assessment_results(user_id)
        else:
            print("Invalid choice.")

In [ ]:
# Verify the database structure
def verify_database_schema():
    # Connect to the database
    conn = sqlite3.connect('./database/app_db.db')
    cursor = conn.cursor()
    
    # Check users table schema
    cursor.execute("PRAGMA table_info(users)")
    users_columns = cursor.fetchall()
    print("Users table columns:")
    for col in users_columns:
        print(f"  {col[1]} ({col[2]})")
    
    # Check conversations table schema
    cursor.execute("PRAGMA table_info(conversations)")
    conversations_columns = cursor.fetchall()
    print("\nConversations table columns:")
    for col in conversations_columns:
        print(f"  {col[1]} ({col[2]})")
    
    conn.close()

verify_database_schema()

# Agent Memory Database Setup

In [ ]:
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.memory.v2.memory import Memory
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# Initialize Agno's native SQLite memory database for conversation context
memory_db = SqliteMemoryDb(
    table_name="agent_memories",
    db_file="./database/agent_memory.db"
)

# Create Agno Memory instance for conversation context
agent_memory = Memory(
    model=OpenAIChat(id="gpt-4o"),
    db=memory_db,
    delete_memories=False,
    clear_memories=False,
)

print("Agent memory database setup complete")

# Create User Authentication and Session Management

In [ ]:
import uuid

class SessionManager:
    def __init__(self):
        self.app_engine = create_engine('sqlite:///./database/app_db.db')
        self.UserSession = sessionmaker(bind=self.app_engine)
        self.active_sessions = {}
        
    def authenticate_user(self, username, password):
        """Authenticate user against the database"""
        session = self.UserSession()
        try:
            user = session.query(User).filter_by(username=username, password=password).first()
            if user:
                # Create a new session
                session_id = str(uuid.uuid4())
                self.active_sessions[session_id] = {
                    'user_id': user.id,
                    'username': user.username,
                    'created_at': datetime.now(),
                    'last_activity': datetime.now()
                }
                print(f"Authentication successful for {username}")
                return session_id
            else:
                print("Authentication failed")
                return None
        except Exception as e:
            print(f"Authentication error: {e}")
            return None
        finally:
            session.close()
    
    def validate_session(self, session_id):
        """Validate if session is still active"""
        if session_id in self.active_sessions:
            # Update last activity
            self.active_sessions[session_id]['last_activity'] = datetime.now()
            return True
        return False
    
    def get_user_from_session(self, session_id):
        """Get user info from session"""
        if self.validate_session(session_id):
            return self.active_sessions[session_id]
        return None

# Initialize session manager
session_manager = SessionManager()

# Test authentication
test_session_id = session_manager.authenticate_user("Nayeem", "password")
print(f"Test session ID: {test_session_id}")

# Agent Setup with Structured Output and Tool

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools import tool
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.memory.v2.memory import Memory
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os

# Create database directory if it doesn't exist
os.makedirs('./database', exist_ok=True)

# Initialize Agno's native SQLite memory database
memory_db = SqliteMemoryDb(
    table_name="agent_memories",
    db_file="./database/agent_memory.db"
)

# Create Agno Memory instance
agent_memory = Memory(
    model=OpenAIChat(id="gpt-4o"),
    db=memory_db,
    delete_memories=False,
    clear_memories=False,
)

# Create the store_to_db tool with proper error handling
@tool
def store_to_db_tool(question: str, answer: str, positive_traits: dict, negative_traits: dict):
    """Stores personality assessment data to the app database."""
    try:
        # Get user info from session manager
        user_info = session_manager.get_user_from_session(test_session_id)
        if not user_info:
            return "Session invalid or expired. Please authenticate again."
        
        user_id = user_info['user_id']
        
        # Create a new app session for thread safety
        app_engine = create_engine('sqlite:///./database/app_db.db')
        Session = sessionmaker(bind=app_engine)
        app_session = Session()
        
        # Store in app_db conversations table
        new_conversation = Conversation(
            user_id=user_id,
            session_id=test_session_id,
            question=question,
            answer=answer,
            positive_traits=positive_traits,
            negative_traits=negative_traits
        )
        
        app_session.add(new_conversation)
        app_session.commit()
        app_session.close()
        
        # Also store in Agno's memory for conversation context
        memory_content = f"User displayed positive traits: {positive_traits} and negative traits: {negative_traits} in response to: {question}"
        agent_memory.add(
            content=memory_content,
            metadata={
                "type": "personality_assessment",
                "question": question,
                "user_id": user_id,
                "session_id": test_session_id
            }
        )
        
        return "Data stored successfully in both app database and agent memory."
    except Exception as e:
        app_session.rollback()
        app_session.close()
        return f"Error storing data: {str(e)}"

# Create the ConversationAgent
conversation_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="""You are a personality test expert. Your goal is to ask users questions to identify their personality traits. 
    Ask one question at a time and wait for the user's response.
    After getting a complete answer, analyze it to extract positive and negative personality traits.
    Then use the store_to_db_tool to save the data before moving to the next question.
    
    Important: You MUST call the store_to_db_tool after each complete answer with these exact parameters:
    - question: The exact question you asked
    - answer: The user's complete response
    - positive_traits: A dictionary like {'trait_name': score} where scores are 0-100
    - negative_traits: A dictionary like {'trait_name': score} where scores are 0-100
    
    Example: store_to_db_tool(
        question="How do you handle stress?",
        answer="I usually stay calm and focused",
        positive_traits={'calmness': 85, 'focus': 75},
        negative_traits={'anxiety': 25}
    )""",
    tools=[store_to_db_tool],
    markdown=True,
    memory=agent_memory,
    enable_agentic_memory=True
)

print("Agent created successfully with proper tool configuration")

# Conversation Loop Implementation

In [ ]:
# Initialize conversation
messages = [
    {"role": "system", "content": conversation_agent.description}
]

print("Starting personality test conversation...")

# Get initial response
try:
    response = conversation_agent.run("Start the personality test with the first question.")
    messages.append({"role": "assistant", "content": response.content})
    print("Agent:", response.content)
except Exception as e:
    print(f"Error starting conversation: {e}")
    fallback_msg = "Hello! Let's start with your personality assessment. How do you typically handle stressful situations?"
    messages.append({"role": "assistant", "content": fallback_msg})
    print("Agent:", fallback_msg)

# Conversation loop
while True:
    try:
        # Get user input
        user_input = input("You: ")
        
        if user_input.lower() in ['exit', 'quit', 'stop', 'end']:
            print("Ending conversation. Thank you for your time!")
            break
        
        # Add user message to history
        messages.append({"role": "user", "content": user_input})
        
        # Get agent response
        response = conversation_agent.run(messages=messages)
        
        # Extract agent response content
        agent_response = response.content
        messages.append({"role": "assistant", "content": agent_response})
        print("Agent:", agent_response)
        
        # Handle tool calls using Agno's recommended approach
        if hasattr(response, 'tools') and response.tools:
            for tool_call in response.tools:
                try:
                    # Use the correct method for tool execution
                    if hasattr(tool_call, 'run'):
                        tool_result = tool_call.run()
                    elif hasattr(tool_call, 'execute'):
                        tool_result = tool_call.execute()
                    else:
                        # Fallback: try to call it directly
                        tool_result = tool_call()
                    print(f"\n[System]: {tool_result}\n")
                except Exception as e:
                    print(f"\n[System]: Tool execution error - {e}\n")
                    # Log the tool call for debugging
                    print(f"Tool call object: {tool_call}")
                    print(f"Tool call type: {type(tool_call)}")
            
    except Exception as e:
        print(f"Error in conversation: {e}")
        error_msg = "I apologize for the technical issue. Let's continue our conversation."
        messages.append({"role": "assistant", "content": error_msg})
        print("Agent:", error_msg)